# Matched pair AB Test

In this notebook we will identify 2 control units for each of the 10 treatment units. We are given 2 files:

newproductcontroldata.csv

grapeproducttreatmentstores.csv

The treatment units are stored in grapeproducttreatmentstores.csv

In [1]:
#import necessary packages:

import pandas as pd
import numpy as np
from scipy import spatial

Read the files to data frames:

In [44]:
treatment_df = pd.read_csv("./downloads/grapeproducttreatmentstores.csv")
control_df = pd.read_csv("./downloads/newproductcontroldata.csv")

display sample rows:

In [50]:
treatment_df

,Store ID,City,State,Zip Code,Category Sales,Product Count,Size
0,332,La Mesa,CA,91942,56792.96,5,Large
1,944,Palmdale,CA,93551,41.50,1,Large
2,1181,Lake Elsinore,CA,92532,1514.11,4,Large
3,2696,Redondo Beach,CA,90278,2006.88,4,Large
4,2717,Thousand Oaks,CA,91320,80.04,2,Large
5,3060,San Francisco,CA,94111,26467.99,7,Large
6,5019,Napa,CA,94558,16744.96,5,Large
7,5051,Simi Valley,CA,93065,1378.85,3,Large
8,5395,Los Angeles,CA,90079,5669.56,5,Large
9,5834,Fremont,CA,94539,19064.48,6,Large


In [51]:
control_df.head()

,Store ID,City,State,Zip Code,Category Sales,Product Count,Size
103,104,Bakersfield,CA,93311,666.18,5,Large
104,105,Canoga Park,CA,91303,868.80,3,Large
105,106,Colma,CA,94014,11682.08,5,Large
106,107,Costa Mesa,CA,92626,58509.72,5,Large
107,108,Dublin,CA,94568,4601.87,4,Large


We do not need City, Zip, Size columns. Also we do not need any other state values except the state of CA in treatment_df. We identified State, Product Count and Category Sales as control variables. Since the treatment units all have CA as the state we are dropping that.

In [52]:
treatment_df = treatment_df[treatment_df['State'] == 'CA']
control_df = control_df[control_df['State'] == 'CA']

In [56]:
treatment_df

,Store ID,City,State,Zip Code,Category Sales,Product Count,Size
0,332,La Mesa,CA,91942,56792.96,5,Large
1,944,Palmdale,CA,93551,41.50,1,Large
2,1181,Lake Elsinore,CA,92532,1514.11,4,Large
3,2696,Redondo Beach,CA,90278,2006.88,4,Large
4,2717,Thousand Oaks,CA,91320,80.04,2,Large
5,3060,San Francisco,CA,94111,26467.99,7,Large
6,5019,Napa,CA,94558,16744.96,5,Large
7,5051,Simi Valley,CA,93065,1378.85,3,Large
8,5395,Los Angeles,CA,90079,5669.56,5,Large
9,5834,Fremont,CA,94539,19064.48,6,Large


In [66]:
control_df.shape

(913, 7)

In [65]:
treatment_df.shape

(10, 7)

Dropping unnecessary columns

In [102]:
treatment_product_count = treatment_df['Product Count']
treatment_category_sales = treatment_df['Category Sales']
treatment_category_store_ID = treatment_df['Store ID']

control_product_count = control_df['Product Count']
control_category_sales = control_df['Category Sales']
control_category_store_ID = control_df['Store ID']

#DO NOT SCALE. As this is NOT giving correct results

#treatment_product_count = (treatment_product_count - treatment_product_count.min())/(treatment_product_count.max() - treatment_product_count.min())
#treatment_category_sales = (treatment_category_sales - treatment_category_sales.min())/(treatment_category_sales.max() - treatment_category_sales.min())

#control_product_count = (control_product_count - control_product_count.min())/(control_product_count.max() - control_product_count.min())
#control_category_sales = (control_category_sales - control_category_sales.min())/(control_category_sales.max() - control_category_sales.min())

In [94]:
len(treatment_product_count)

10

In [95]:
len(control_product_count)

913

In [96]:
tree = spatial.KDTree(list(zip(control_product_count, control_category_sales)))
tree.data
#list(zip(treatment_product_count, treatment_category_sales))

array([[5.000000e+00, 6.661800e+02],
       [3.000000e+00, 8.688000e+02],
       [5.000000e+00, 1.168208e+04],
       ...,
       [6.000000e+00, 8.387900e+03],
       [5.000000e+00, 2.537506e+04],
       [5.000000e+00, 3.799680e+03]])

In [97]:
pts = list(zip(treatment_product_count, treatment_category_sales))
pts

[(5, 56792.96),
 (1, 41.5),
 (4, 1514.11),
 (4, 2006.88),
 (2, 80.04),
 (7, 26467.99),
 (5, 16744.96),
 (3, 1378.85),
 (5, 5669.56),
 (6, 19064.48)]

In [98]:
tree.query(pts, k=2,p=2)[1]

array([[ 55, 781],
       [201, 674],
       [241, 359],
       [390,  28],
       [411, 513],
       [494, 787],
       [681, 672],
       [713, 239],
       [798, 484],
       [832, 605]])

In [101]:
k = 0
for i,j,l in tree.query(pts, k=3)[1]:
    #print(i,j)
    print(treatment_df.iloc[k])
    k += 1
    print(control_df[control_df['Store ID'] == list(control_category_store_ID)[j]])
    print(control_df[control_df['Store ID'] == list(control_category_store_ID)[l]])    
    print("\n\n")
    #print(list(control_category_store_ID)[i],list(control_category_store_ID)[j])

Store ID              332
City              La Mesa
State                  CA
Zip Code            91942
Category Sales      56793
Product Count           5
Size                Large
Name: 0, dtype: object
      Store ID     City State  Zip Code  Category Sales  Product Count   Size
5377      5378  Anaheim    CA     92807        56371.04              6  Small
      Store ID             City State  Zip Code  Category Sales  \
3955      3956  North Highlands    CA     95660        57243.36   

      Product Count    Size  
3955              5  Medium  



Store ID               944
City              Palmdale
State                   CA
Zip Code             93551
Category Sales        41.5
Product Count            1
Size                 Large
Name: 1, dtype: object
      Store ID  City State  Zip Code  Category Sales  Product Count   Size
5011      5012  Lodi    CA     95242           40.58              2  Large
      Store ID      City State  Zip Code  Category Sales  Product Count   Size
